# Imports (compatible for Python 2 and Python 3)

In [6]:
try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd
import matplotlib.pylab as plt

%matplotlib inline

In [2]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

### Task1:  Familiarize with SQL Clauses

In [18]:
# Query data from citibike database

query = '''SELECT * FROM fb55.citibike; '''

table= get_data(query)

table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


In [28]:
# Task1.1 : 
# Sort data by start_station_id, tripduration
# - Only checking trips with duration <= 3 hours 

query = '''SELECT * FROM fb55.citibike
            WHERE tripduration<=180
            ORDER BY start_station_id ASC, tripduration ASC;
            '''

table1_1 = get_data(query)

table1_1.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980.0,2
1,NaN,Franklin St & W Broadway,NaN,44151,-74.006667,149,2015-02-07 17:27:00+00,2015-02-07 17:29:00+00,79,44145,40.719116,257,Lispenard St & Broadway,40.719392,-74.002472,15621,Subscriber,1973.0,1
2,NaN,St James Pl & Pearl St,NaN,30674,-74.000165,138,2015-02-05 22:22:00+00,2015-02-05 22:25:00+00,82,30628,40.711174,296,Division St & Bowery,40.714131,-73.997047,17620,Subscriber,1989.0,1
3,NaN,St James Pl & Pearl St,NaN,4787,-74.000165,143,2015-02-01 17:12:00+00,2015-02-01 17:14:00+00,82,4719,40.711174,296,Division St & Bowery,40.714131,-73.997047,18634,Subscriber,1989.0,1
4,NaN,St James Pl & Pearl St,NaN,3787,-74.000165,144,2015-02-01 15:34:00+00,2015-02-01 15:36:00+00,82,3720,40.711174,2009,Catherine St & Monroe St,40.711174,-73.996826,20906,Subscriber,1984.0,1


In [148]:
# Task1_2: Only show the top/last 10 records (aka head and tail in SQL) 

In [149]:
# Head
query = '''SELECT * FROM fb55.citibike
            ORDER BY cartodb_id ASC
            LIMIT 10; '''

table1_2_h = get_data(query)

table1_2_h.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,8 Ave & W 31 St,NaN,70,-73.994811,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,1,40.750450,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978,2
1,NaN,E 17 St & Broadway,NaN,71,-73.990093,379,2015-02-01 00:00:00+00,2015-02-01 00:07:00+00,497,2,40.737050,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993,1
2,NaN,Grand Army Plaza & Central Park S,NaN,72,-73.973715,2474,2015-02-01 00:01:00+00,2015-02-01 00:42:00+00,281,3,40.764397,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969,2
3,NaN,6 Ave & Broome St,NaN,73,-74.004704,818,2015-02-01 00:01:00+00,2015-02-01 00:15:00+00,2004,4,40.724399,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985,2
4,NaN,Lawrence St & Willoughby St,NaN,74,-73.986317,544,2015-02-01 00:01:00+00,2015-02-01 00:10:00+00,323,5,40.692362,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,19868,Subscriber,1957,1
5,NaN,Willoughby Ave & Walworth St,NaN,75,-73.953820,717,2015-02-01 00:02:00+00,2015-02-01 00:14:00+00,373,6,40.693317,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,15854,Subscriber,1979,1
6,NaN,W 56 St & 6 Ave,NaN,76,-73.977225,1306,2015-02-01 00:04:00+00,2015-02-01 00:26:00+00,352,7,40.763406,504,1 Ave & E 15 St,40.732219,-73.981656,15173,Subscriber,1983,1
7,NaN,E 4 St & 2 Ave,NaN,77,-73.989780,913,2015-02-01 00:04:00+00,2015-02-01 00:19:00+00,439,8,40.726281,116,W 17 St & 8 Ave,40.741776,-74.001497,17862,Subscriber,1955,1
8,NaN,Washington Pl & Broadway,NaN,78,-73.994046,759,2015-02-01 00:04:00+00,2015-02-01 00:17:00+00,335,9,40.729039,2012,E 27 St & 1 Ave,40.739445,-73.976806,21183,Subscriber,1985,2
9,NaN,Greenwich Ave & 8 Ave,NaN,79,-74.002638,585,2015-02-01 00:05:00+00,2015-02-01 00:15:00+00,284,10,40.739017,444,Broadway & W 24 St,40.742354,-73.989151,14843,Subscriber,1982,1


In [33]:
# Tail
query = '''SELECT * FROM fb55.citibike
            ORDER BY cartodb_id DESC
            LIMIT 10; '''

table1_2_t = get_data(query)

table1_2_t.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 4 St & 2 Ave,NaN,46199,-73.989780,917,2015-02-07 23:59:00+00,2015-02-08 00:15:00+00,439,46200,40.726281,417,Barclay St & Church St,40.712912,-74.010202,20998,Subscriber,1965,2
1,NaN,Carmine St & 6 Ave,NaN,46198,-74.002150,548,2015-02-07 23:58:00+00,2015-02-08 00:08:00+00,368,46199,40.730386,334,W 20 St & 7 Ave,40.742388,-73.997262,19540,Subscriber,1983,2
2,NaN,5 Ave & E 29 St,NaN,46197,-73.986831,392,2015-02-07 23:57:00+00,2015-02-08 00:03:00+00,474,46198,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,15545,Subscriber,1986,1
3,NaN,5 Ave & E 29 St,NaN,46196,-73.986831,428,2015-02-07 23:57:00+00,2015-02-08 00:04:00+00,474,46197,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,16395,Subscriber,1986,2
4,NaN,W 20 St & 8 Ave,NaN,46195,-74.000040,689,2015-02-07 23:57:00+00,2015-02-08 00:08:00+00,470,46196,40.743453,325,E 19 St & 3 Ave,40.736245,-73.984738,15585,Subscriber,1953,1
5,NaN,1 Ave & E 44 St,NaN,46194,-73.969053,1422,2015-02-07 23:57:00+00,2015-02-08 00:20:00+00,455,46195,40.750020,265,Stanton St & Chrystie St,40.722293,-73.991475,20184,Subscriber,1960,2
6,NaN,E 20 St & 2 Ave,NaN,46193,-73.982050,993,2015-02-07 23:56:00+00,2015-02-08 00:13:00+00,461,46194,40.735877,295,Pike St & E Broadway,40.714067,-73.992939,16722,Subscriber,1974,1
7,NaN,Warren St & Church St,NaN,46192,-74.009106,1165,2015-02-07 23:56:00+00,2015-02-08 00:15:00+00,152,46193,40.714740,325,E 19 St & 3 Ave,40.736245,-73.984738,16978,Subscriber,1959,1
8,NaN,W 47 St & 10 Ave,NaN,46191,-73.993012,1248,2015-02-07 23:56:00+00,2015-02-08 00:17:00+00,495,46192,40.762699,432,E 7 St & Avenue A,40.726218,-73.983799,16300,Subscriber,1984,1
9,NaN,W 17 St & 8 Ave,NaN,46190,-74.001497,306,2015-02-07 23:55:00+00,2015-02-08 00:01:00+00,116,46191,40.741776,494,W 26 St & 8 Ave,40.747348,-73.997236,14736,Subscriber,1983,2


In [150]:
# Task1_3:  List all unique start_station_id values

query = '''SELECT DISTINCT START_STATION_ID FROM fb55.citibike
            ORDER BY START_STATION_ID ASC; '''

table1_3 = get_data(query)

table1_3.head()

,start_station_id,Unnamed: 1
0,72,NaN
1,79,NaN
2,82,NaN
3,83,NaN
4,116,NaN


In [151]:
# Task 1_4: Aggregation functions:
#            - Count the number of trips (aka wc -l in SQL)
#            - Find the average/min/max trip duration

In [152]:
# Count the number of trips (aka wc -l in SQL)

query = '''SELECT COUNT(cartodb_id)
            FROM fb55.citibike; '''

table1_4_a = get_data(query)

table1_4_a.head()

,count,Unnamed: 1
0,46200,NaN


In [153]:
# Find minimal, maximal and average trip duration hint: min(), max(), avg()

query = '''SELECT MIN(tripduration) AS min_trip_duration, MAX(tripduration) AS max_trip_duration, AVG(tripduration) AS avg_trip_duration
            FROM fb55.citibike; '''

table1_4_b = get_data(query)

table1_4_b.head()

,min_trip_duration,max_trip_duration,avg_trip_duration
0,60,43016,675.865823


### Task 2 : Working with date/time

In [154]:
# Task2_1: Selecting trips started on Feb-02-2015 only ;

query = '''SELECT * FROM fb55.citibike
WHERE EXTRACT(DAY FROM starttime)=2 AND EXTRACT(MONTH FROM starttime)=2 AND EXTRACT(YEAR FROM starttime)=2015; '''

table2_1 = get_data(query)

table2_1.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [155]:
# Task2_2: Selecting trips started on the weekends
# - What are average trip duration during weekends?

query = '''SELECT AVG(tripduration) AS avg_tripduration_weekends
            FROM fb55.citibike
            WHERE EXTRACT(dow FROM starttime)=0 OR EXTRACT(dow FROM starttime)=6; '''

table2_2 = get_data(query)

table2_2

,avg_tripduration_weekends,Unnamed: 1
0,662.942181,NaN


In [156]:
# Task2_3: Can we do the same for weekday?
query = '''SELECT AVG(tripduration) AS avg_tripduration_weekday
            FROM fb55.citibike
            WHERE EXTRACT(dow FROM starttime)>0 AND EXTRACT(dow FROM starttime)<6; '''

table2_3 = get_data(query)

table2_3.head()

,avg_tripduration_weekday,Unnamed: 1
0,681.052292,NaN


### Task3: Working with Space 

In [157]:
# Task 3_1: Showing the list of start station locations
#            - Using GROUP BY 

query = '''SELECT start_station_id, start_station_latitude as latitude , start_station_longitude as longitude
            FROM fb55.citibike 
            GROUP BY start_station_id,start_station_latitude,start_station_longitude;'''

table3_1 = get_data(query)

table3_1.head()

,start_station_id,latitude,longitude
0,349,40.718502,-73.983299
1,470,40.743453,-74.000040
2,485,40.750380,-73.983390
3,335,40.729039,-73.994046
4,267,40.750977,-73.987654


In [158]:
# Task 3_2: Showing the number of trips started per station 

query = '''SELECT start_station_id, COUNT(start_station_id)
            FROM fb55.citibike
            GROUP BY start_station_id; '''

table3_2 = get_data(query)

table3_2.head()

,start_station_id,count
0,120,17
1,285,373
2,251,194
3,195,137
4,453,218


In [159]:
# Task 3_3: Showing the number of trips started per station
# but only for stations within 500m of Time Square!
#- The coordinates of Time Square is (40.7577,-73.9857)
#ST_MakePoint(start_station_latitude,start_station_longitude)

query = ''' SELECT start_station_id, count
            FROM ( SELECT start_station_id, start_station_latitude,start_station_longitude, ST_Distance_Sphere(ST_MakePoint(start_station_latitude,start_station_longitude), ST_GeomFromText('POINT(40.7577 -73.9857)',4326)), count(start_station_id)
                    FROM fb55.citibike
                    GROUP BY start_station_id, start_station_latitude, start_station_longitude) assome
                    WHERE st_distance_sphere < 500;'''

table3_3 = get_data(query)

table3_3.head()

,start_station_id,count
0,485,157
1,267,177
2,449,107
3,2021,141
4,505,296


### Task 4: Putting it all together

In [139]:
# Task 4_1: Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare! 

query = ''' SELECT start_station_id, tripduration
            FROM ( SELECT start_station_id, start_station_latitude,start_station_longitude, ST_Distance_Sphere(ST_MakePoint(start_station_latitude,start_station_longitude), ST_GeomFromText('POINT(40.7577 -73.9857)',4326)), count(start_station_id), avg(tripduration) as tripduration
                    FROM fb55.citibike
                    WHERE EXTRACT(dow FROM starttime)=0 OR EXTRACT(dow FROM starttime)=6
                    GROUP BY start_station_id, start_station_latitude, start_station_longitude) assome
                    WHERE st_distance_sphere < 500
                    ORDER BY tripduration DESC
                    LIMIT 1;'''

table4_1 = get_data(query)

table4_1.head()

,start_station_id,tripduration
0,173,1010.104167


In [145]:
# Extra: create lines for trips started from stations within 500m of Times
# Squares and lasted less than 2 hours. The number of trips per each
# pair of stations are output as attributes of these lines. 

query = '''SELECT start_station_id, end_station_id, ST_MakeLine(ST_MakePoint(start_station_longitude,start_station_latitude),ST_MakePoint(end_station_longitude,end_station_latitude)) as tripline 
            FROM fb55.citibike 
            WHERE ((ST_Distance_Sphere(ST_MakePoint(-73.9857,40.7577),ST_MakePoint(start_station_longitude,start_station_latitude))) <500) 
                    AND tripduration< 500
            GROUP BY start_station_id ,end_station_id,start_station_longitude,start_station_latitude, end_station_longitude,end_station_latitude;'''

table4_2 = get_data(query)

table4_2.head()

,start_station_id,end_station_id,tripline
0,493,137,010200000002000000860DC305E87E52C0058E5FD3DE60...
1,465,537,010200000002000000BE51C821247F52C021994848A860...
2,173,495,010200000002000000385762D8007F52C0BF6DBFDF5C61...
3,465,533,010200000002000000BE51C821247F52C021994848A860...
4,524,522,01020000000200000046E82D3FEC7E52C08A99B7C9AC60...
